In [22]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,LSTM,BatchNormalization,Dropout,MaxPool2D,Flatten,Dense,Reshape
from tensorflow.keras import Sequential
from tensorflow.data import Dataset
import numpy as np
import csv

In [34]:
feature_extraction_layers = [
    
    Conv2D(16,(3,3),input_shape=(15,1025,151),activation="relu",name="Convolution_Layer_1"),
    MaxPool2D((2,2)),
    BatchNormalization(),
    Conv2D(32,(2,2),activation="relu",name="Convolution_Layer_2"),
    MaxPool2D((2,2)),
    BatchNormalization(),
    Dropout(0.3),
    Conv2D(64,(2,2),activation="relu",name="Convolution_Layer_3"),
    BatchNormalization(),
    Reshape((128,127))
    
]

In [47]:
classification_layers = [
      
    LSTM(128),
    Dropout(0.5),
    Reshape((128,1)),
    LSTM(128),
    Dense(30,activation="relu"),
    Dense(1,activation="sigmoid"),
#     Reshape((1,2))
    
]
full_layers = [layer for layer in feature_extraction_layers]
full_layers+= [layer for layer in classification_layers]

In [48]:
model = Sequential(full_layers)


In [49]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Convolution_Layer_1 (Conv2D) (None, 13, 1023, 16)      21760     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 6, 511, 16)        0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 6, 511, 16)        64        
_________________________________________________________________
Convolution_Layer_2 (Conv2D) (None, 5, 510, 32)        2080      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 2, 255, 32)        0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 2, 255, 32)        128       
_________________________________________________________________
dropout_5 (Dropout)          (None, 2, 255, 32)       

In [53]:

def create_dataset(folder,csv_file):
    features=[]
    labels=[]
    with open(folder+csv_file) as csv_file:
        reader =  csv.reader(csv_file,delimiter=",")
        for row in reader:
            features.append(np.load(folder+row[0]))
            labels.append(float(row[1]))
    return features,labels

    
def prepare_dataset(folder,csv_file):
    reshaped_features= []
    features,labels= create_dataset(folder,csv_file)
    for feature in features:
        reshaped_features.append(feature.reshape(1,15,1025,151))
    reshaped_features=np.array(reshaped_features)
    labels = np.array(labels)
    labels=labels.reshape(-1,1)
#     labels = tf.one_hot(labels,2)
    dataset = Dataset.from_tensor_slices((reshaped_features,labels))
    dataset = dataset.repeat(40)
    return dataset,reshaped_features,labels

In [54]:
dataset,reshaped_features,labels = prepare_dataset("./training-preprocessed/","training-dataset.csv")
# print(labels)
validation_dataset,validation_features,_validation_labels = prepare_dataset("./preprocessed-validation-data/","validation-dataset.csv")


In [ ]:
model.compile(loss="binary_crossentropy",optimizer=tf.keras.optimizers.Adam(0.001),metrics=["accuracy"])
model.fit(dataset,epochs=10,validation_data=validation_dataset)

Epoch 1/10
1129/2480 [============>.................] - ETA: 2:46 - loss: 0.6446 - accuracy: 0.7836

In [30]:
test=np.load("./training-preprocessed/Patient_1_preictal_segment_0002_21.npy")
result=model.predict(test.reshape(1,15,1025,151))
print(result)

FileNotFoundError: [Errno 2] No such file or directory: './training-preprocessed/Patient_1_preictal_segment_0002_21.npy'

In [278]:
test=np.load("./training-preprocessed/Patient_1_interictal_segment_0001_1.npy")
result=model.predict(test.reshape(1,15,1025,151))
print(result)

[[0.6268092]]
